In [1]:
import numpy as np
import pandas as pd
import pywt
%matplotlib qt
#%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
ticker = 'BSX'
df = pd.read_csv('./{}.csv'.format(ticker))
close = df.Close
close.head()

0    11.50000
1    11.68750
2    12.53125
3    12.87500
4    12.40625
Name: Close, dtype: float64

In [3]:
scaler = MinMaxScaler(feature_range=(-1,1))

split = int(len(close)*.8)
train = close[:split]
test = close[split:]

train_sc = scaler.fit_transform(np.array(train).reshape(-1,1)).reshape(train.shape)
test_sc = scaler.fit_transform(np.array(test).reshape(-1,1)).reshape(test.shape)

In [4]:
mode = 'garotte'
wavelet = 'haar'

#denoise train
ca, cd = pywt.dwt(train_sc, wavelet=wavelet)
thr = np.std(train_sc)/32
cat = pywt.threshold(ca, thr, mode=mode)
cdt = pywt.threshold(cd, thr, mode=mode)
train_rc = pywt.idwt(cat, cdt, 'haar')

#denoise test
ca, cd = pywt.dwt(test_sc, wavelet=wavelet)
thr = np.std(test_sc)/32
cat = pywt.threshold(ca, thr, mode=mode)
cdt = pywt.threshold(cd, thr, mode=mode)
test_rc = pywt.idwt(cat, cdt, 'haar')

plt.plot(test_sc, color='red')
plt.plot(test_rc)
plt.show()

In [5]:
X_train = train_rc[:-1]
y_train = train_rc[1:]

X_test = test_rc[:-1]
y_test = test_rc[1:]

In [11]:
#Simple ANN for time series
nn_model = Sequential()
nn_model.add(Dense(12, input_dim=1, activation='relu'))
nn_model.add(Dense(1))
nn_model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
history = nn_model.fit(X_train,
                       y_train,
                       epochs=100,
                       batch_size=1,
                       verbose=1,
                       #callbacks=[early_stop],
                       shuffle=False)

Epoch 1/100
3889/3889 [==============================] - 3s 883us/step - loss: 0.0094 - mean_squared_error: 0.0094 - mean_absolute_error: 0.0388
Epoch 2/100
3889/3889 [==============================] - 3s 814us/step - loss: 7.5669e-04 - mean_squared_error: 7.5669e-04 - mean_absolute_error: 0.0166
Epoch 3/100
3889/3889 [==============================] - 3s 812us/step - loss: 5.9690e-04 - mean_squared_error: 5.9690e-04 - mean_absolute_error: 0.0149
Epoch 4/100
3889/3889 [==============================] - 3s 825us/step - loss: 5.1630e-04 - mean_squared_error: 5.1630e-04 - mean_absolute_error: 0.0140
Epoch 5/100
3889/3889 [==============================] - 3s 832us/step - loss: 4.6848e-04 - mean_squared_error: 4.6848e-04 - mean_absolute_error: 0.0134
Epoch 6/100
3889/3889 [==============================] - 3s 837us/step - loss: 4.3878e-04 - mean_squared_error: 4.3878e-04 - mean_absolute_error: 0.0129
Epoch 7/100
3889/3889 [==============================] - 3s 820us/step - loss: 4.2622e-04 

3889/3889 [==============================] - 3s 793us/step - loss: 4.1592e-04 - mean_squared_error: 4.1592e-04 - mean_absolute_error: 0.0121
Epoch 55/100
3889/3889 [==============================] - 3s 795us/step - loss: 4.1578e-04 - mean_squared_error: 4.1578e-04 - mean_absolute_error: 0.0121
Epoch 56/100
3889/3889 [==============================] - 3s 792us/step - loss: 4.1565e-04 - mean_squared_error: 4.1565e-04 - mean_absolute_error: 0.0121
Epoch 57/100
3889/3889 [==============================] - 3s 807us/step - loss: 4.1541e-04 - mean_squared_error: 4.1541e-04 - mean_absolute_error: 0.0121
Epoch 58/100
3889/3889 [==============================] - 3s 794us/step - loss: 4.1523e-04 - mean_squared_error: 4.1523e-04 - mean_absolute_error: 0.0121
Epoch 59/100
3889/3889 [==============================] - 3s 799us/step - loss: 4.1509e-04 - mean_squared_error: 4.1509e-04 - mean_absolute_error: 0.0121
Epoch 60/100
3889/3889 [==============================] - 3s 798us/step - loss: 4.1497e-0

In [12]:
#plot metrics
plt.subplot(2,1,1)
plt.plot(history.history['loss'])
plt.title('loss')

plt.subplot(2,1,2)
plt.title('mae')
plt.plot(history.history['mean_absolute_error'])

In [14]:
y_test_pred = nn_model.predict(X_test)
y_train_pred = nn_model.predict(X_train)

print('R2 score on the train set: {:0.3f}'.format(r2_score(y_train, y_train_pred)))
print('R2 score on the test set: {:0.3f}'.format(r2_score(y_test, y_test_pred)))
plt.plot(y_test, color='red')
plt.plot(y_test_pred)
plt.title('Test set')
plt.legend(['actual', 'predicted'])

R2 score on the train set: 0.926
R2 score on the test set: 0.951


In [15]:
#print(train_sc.shape, train_rc.shape)
train_rc_df = pd.DataFrame(train_rc[:-1], columns=['Y'], index=train.index)
test_rc_df = pd.DataFrame(test_rc[:-1], columns=['Y'], index=test.index)

for s in range(1,2):
    train_rc_df['X_{}'.format(s)] = train_rc_df['Y'].shift(s)
    test_rc_df['X_{}'.format(s)] = test_rc_df['Y'].shift(s)
    
X_train = train_rc_df.dropna().drop('Y', axis=1)
y_train = train_rc_df.dropna().drop('X_1', axis=1)

X_test = test_rc_df.dropna().drop('Y', axis=1)
y_test = test_rc_df.dropna().drop('X_1', axis=1)

X_train = X_train.values
y_train = y_train.values

X_test = X_test.values
y_test = y_test.values

X_train_lmse = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lmse = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print('Train shape: ', X_train_lmse.shape)
print('Test shape: ', X_test_lmse.shape)

Train shape:  (3888, 1, 1)
Test shape:  (972, 1, 1)


In [16]:
#LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(7, input_shape=(1, X_train_lmse.shape[1]),
                    activation='relu',
                    kernel_initializer='lecun_uniform',
                    return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','mae'])

early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
history_lstm_model = lstm_model.fit(X_train_lmse,
                                    y_train,
                                    epochs=100,
                                    batch_size=1,
                                    verbose=1,
                                    #callbacks=[early_stop],
                                    shuffle=False)

Epoch 1/100
3888/3888 [==============================] - 8s 2ms/step - loss: 0.0157 - mean_squared_error: 0.0157 - mean_absolute_error: 0.0514
Epoch 2/100
3888/3888 [==============================] - 8s 2ms/step - loss: 5.0707e-04 - mean_squared_error: 5.0707e-04 - mean_absolute_error: 0.0140
Epoch 3/100
3888/3888 [==============================] - 7s 2ms/step - loss: 4.3116e-04 - mean_squared_error: 4.3116e-04 - mean_absolute_error: 0.0128
Epoch 4/100
3888/3888 [==============================] - 7s 2ms/step - loss: 4.1206e-04 - mean_squared_error: 4.1206e-04 - mean_absolute_error: 0.0124
Epoch 5/100
3888/3888 [==============================] - 7s 2ms/step - loss: 4.0204e-04 - mean_squared_error: 4.0204e-04 - mean_absolute_error: 0.0122
Epoch 6/100
3888/3888 [==============================] - 7s 2ms/step - loss: 3.9601e-04 - mean_squared_error: 3.9601e-04 - mean_absolute_error: 0.0121
Epoch 7/100
3888/3888 [==============================] - 7s 2ms/step - loss: 3.9373e-04 - mean_squared

3888/3888 [==============================] - 9s 2ms/step - loss: 4.5035e-04 - mean_squared_error: 4.5035e-04 - mean_absolute_error: 0.0124
Epoch 56/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5175e-04 - mean_squared_error: 4.5175e-04 - mean_absolute_error: 0.0124
Epoch 57/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5348e-04 - mean_squared_error: 4.5348e-04 - mean_absolute_error: 0.0124
Epoch 58/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5507e-04 - mean_squared_error: 4.5507e-04 - mean_absolute_error: 0.0124
Epoch 59/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5617e-04 - mean_squared_error: 4.5617e-04 - mean_absolute_error: 0.0124
Epoch 60/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5704e-04 - mean_squared_error: 4.5704e-04 - mean_absolute_error: 0.0124
Epoch 61/100
3888/3888 [==============================] - 8s 2ms/step - loss: 4.5786e-04 - mean_squar

In [17]:
y_test_pred = lstm_model.predict(X_test_lmse)
y_train_pred = lstm_model.predict(X_train_lmse)

print('R2 score on the train set: {:0.3f}'.format(r2_score(y_train, y_train_pred)))
print('R2 score on the test set: {:0.3f}'.format(r2_score(y_test, y_test_pred)))
plt.plot(y_test, color='red')
plt.plot(y_test_pred)
plt.title('Test set')
plt.legend(['actual', 'predicted'])

R2 score on the train set: 0.926
R2 score on the test set: 0.942
